Clone MiCE repo:

In [ ]:
!git clone --branch random_with_pos https://github.com/thanoskaravangelis/mice

In [ ]:
%cd mice

Conda installation

In [ ]:
!pip install -r requirements.txt --force-reinstall

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
!wget https://storage.googleapis.com/allennlp-public-models/mice-newsgroups-predictor.tar.gz
!wget https://storage.googleapis.com/allennlp-public-models/mice-newsgroups-editor.pth
!mkdir -p trained_predictors/newsgroups/model
!mkdir -p results/newsgroups/editors/mice
!mv mice-newsgroups-predictor.tar.gz trained_predictors/newsgroups/model/model.tar.gz
!mv mice-newsgroups-editor.pth results/newsgroups/editors/mice/newsgroups_editor.pth

In [ ]:
!wget https://storage.googleapis.com/allennlp-public-models/mice-imdb-predictor.tar.gz
!wget https://storage.googleapis.com/allennlp-public-models/mice-imdb-editor.pth
!mkdir -p trained_predictors/imdb/model
!mkdir -p results/imdb/editors/mice
!mv mice-imdb-predictor.tar.gz trained_predictors/imdb/model/model.tar.gz
!mv mice-imdb-editor.pth results/imdb/editors/mice/imdb_editor.pth

### Load predictors

In [ ]:
from src.utils import *

In [ ]:
predictor = load_predictor("imdb")

In [ ]:
predictor_news = load_predictor("newsgroups")

### Specify paths

In [ ]:
FOLDERS_PATH = '/kaggle/input/mice-pos-adj-not-random/mice/results/imdb/edits'
PICKLE_PATH = '/kaggle/input/pickle-imdb-500/imdb_mice.pickle'
PICKLE_PATH2 = '/kaggle/input/pickle-files-imdb500/imdb_500_adj.pickle'

In [ ]:
import pickle

# utility functions for loading and saving objects
def save_pickle(obj, filename):
    with open(filename,'wb') as f:
        pickle.dump(obj,f)
  
def load_pickle(filename):
    with open(filename,'rb') as f:
        return pickle.load(f)

In [ ]:
adj = load_pickle("/kaggle/input/all-pickle-files/newsgroups_textfooler_ADJ.pickle")
noun = load_pickle("/kaggle/input/all-pickle-files/newsgroups_textfooler_NOUN.pickle")
verb = load_pickle("/kaggle/input/all-pickle-files/newsgroups_textfooler_VERB.pickle")

In [ ]:
adj_list = []
for item in adj:
    if len(item) > 1:
        adj_list.append(item[0][0])
print("ADJ: ", len(adj_list))
noun_list = []
for item in noun: 
    if len(item) > 1:
        noun_list.append(item[0][0])
print("NOUN: ", len(noun_list))
verb_list = []
for item in verb:
    if len(item) > 1:
        verb_list.append(item[0][0])
print("VERB: ", len(verb_list))
all_list = adj_list + noun_list + verb_list
all_set = set(all_list)
print(len(all_set))

#### IMDb
MiCE grad: 331/430 sentences flipped \
MiCE random: 284/430 sentences flipped \
Polyjuice: 391/430 sentences flipped \
Textfooler: 153/430 sentences flipped 

#### Newsgroups
MiCE grad: 769/1000 sentences flipped \
MiCE random: 687/100 \
Polyjuice: 630/1000 \
Textfooler: 642/1000

### Evaluation functions

In [ ]:
def read_edits(path):
    edits = pd.read_csv(path, sep="\t", lineterminator="\n", error_bad_lines=False, warn_bad_lines=True)

    if edits['new_pred'].dtype == pd.np.dtype('float64'):
        edits['new_pred'] = edits.apply(lambda row: str(int(row['new_pred']) if not np.isnan(row['new_pred']) else ""), axis=1)
        edits['orig_pred'] = edits.apply(lambda row: str(int(row['orig_pred']) if not np.isnan(row['orig_pred']) else ""), axis=1)
        edits['contrast_pred'] = edits.apply(lambda row: str(int(row['contrast_pred']) if not np.isnan(row['contrast_pred']) else ""), axis=1)
    else:
        edits['new_pred'].fillna(value="", inplace=True)
        edits['orig_pred'].fillna(value="", inplace=True)
        edits['contrast_pred'].fillna(value="", inplace=True)
    return edits

def get_best_edits(edits):
    """ MiCE writes all edits that are found in Stage 2, 
    but we only want to evaluate the smallest per input. 
    Calling get_sorted_e() """
    return edits[edits['sorted_idx'] == 0]

In [ ]:
def evaluate_edits(edits):
    temp = edits[edits['sorted_idx'] == 0]
    minim = temp['minimality'].mean()
    flipped = temp[temp['new_pred'].astype(str)==temp['contrast_pred'].astype(str)]
    nunique = temp['data_idx'].nunique()
    flip_rate = len(flipped)/nunique
    duration=temp['duration'].mean()
    metrics = {
        "num_total": nunique,
        "num_flipped": len(flipped),
        "flip_rate": flip_rate,
        "minimality": minim,
        "duration": duration,
    }
    for k, v in metrics.items():
        print(f"{k}: \t{round(v, 3)}")
    return metrics

def display_edits(row):
    html_original, html_edited = html_highlight_diffs(row['orig_editable_seg'], row['edited_editable_seg'])
    minim = round(row['minimality'], 3)
    print(f"MINIMALITY: \t{minim}")
    print("")
    display(HTML(html_original))
    display(HTML(html_edited))

def display_classif_results(rows):
    for _, row in rows.iterrows():
      if row['new_contrast_prob_pred']:
        orig_contrast_prob_pred = round(row['orig_contrast_prob_pred'], 3)
        new_contrast_prob_pred = round(row['new_contrast_prob_pred'], 3)
        print("-----------------------")
        print(f"ORIG LABEL: \t{row['orig_pred']}")
        print(f"CONTR LABEL: \t{row['contrast_pred']} (Orig Pred Prob: {orig_contrast_prob_pred})")
        print(f"NEW LABEL: \t{row['new_pred']} (New Pred Prob: {new_contrast_prob_pred})")
        print("")
        display_edits(row)

In [ ]:
import pandas as pd
import sys
sys.path.append("..")
from src.utils import html_highlight_diffs
from IPython.core.display import display, HTML
import numpy as np
from src.utils import load_predictor, get_ints_to_labels

# Get Flip rates from pickle files

In [ ]:
from tqdm import tqdm
import spacy
nlp = spacy.load("en_core_web_sm")
pickle_file = "/kaggle/input/all-pickle-files/newsgroups_mice_gradient.pkl"
edit_list = load_pickle(pickle_file)
total = len(edit_list)
new_list = []
for i in range(1,10):
    flipped_count = 0
    for item in edit_list:
        if len(item) >= i+1:
            flipped_count+=1
        # a way to see which of the non-flipped sentences contain the targeted POS tag
        """if len(item) == i:
            doc = nlp(item[i-1][0])
            nouns = len([word for word in doc if word.pos_=="ADJ"])
            print(f"Adjectives: {nouns}")"""
    flip_rate = flipped_count / total
    print(f"Flip rate at step {i}: {round(flip_rate*100,3)}%")
    print("----"*10)
    total = flipped_count

## with probs

In [ ]:
from tqdm import tqdm
#pickle_file = "/kaggle/input/all-pickle-files/newsgroups_textfooler_VERB.pickle"
import os 
l = [fil for fil in os.listdir("/kaggle/input/all-pickle-files") if "beam" in fil or "greedy" in fil]
for pickle_file in l:
    edit_list = load_pickle("/kaggle/input/all-pickle-files/" + pickle_file)
    new_list = []
    flip_rate = []
    total = len(edit_list)
    for i in range(0,9):
        flipped_count = 0
        total = len([item[0] for item in edit_list if len(item)>i])
        for item in edit_list:
            if len(item) > i+1:
                #for newsgroups
                if (item[i][1] != item[i+1][1]):
                    flipped_count+=1
                """prev_arr = np.array(item[i][1])
                prev_max_idx = np.array(prev_arr).argmax()
                new_arr = np.array(item[i+1][1])
                new_max_idx = np.array(new_arr).argmax()
                if new_max_idx!=prev_max_idx:
                    flipped_count+=1"""
                """#for imdb
                if round(item[i][1][0]) != round(item[i+1][1][0]):
                    flipped_count+=1"""
        flip_rate.append(round((flipped_count / total),4))
        total = flipped_count
    print(pickle_file.split('/')[-1].split('.pickle')[0].split('.pkl')[0], '=', flip_rate)
    

# Create pickle files

# For text fooler


In [ ]:
import pandas as pd
csv = pd.read_csv(FOLDERS_PATH+"/log_roberta_imdb_0.csv")
csv.iloc[0]

In [ ]:
import pandas as pd
from tqdm import tqdm
picklist = []
edits = pd.read_csv(f"{FOLDERS_PATH}/mice_imdb_ADJ_0/edits.csv")
for index, row in tqdm(edits.iterrows()):
  orig_input = row['original_text']
  orig_pred = predictor.predict(str(orig_input))
  orig_pred = add_probs(orig_pred)
  orig_probs = orig_pred['probs'][::-1]
  my_tupe = [orig_input, orig_probs]
  picklist.append([my_tupe])
picklist

In [ ]:
for sublist in tqdm(picklist):
  text = sublist[0][0]
  for i in range(10):
    edits = pd.read_csv(f"/kaggle/input/textfooler-files/log_allennlp_VERB_new_{i}.csv")
    selected_edits = edits.loc[edits["original_text"] == text]
    #print(text)
    orig_pred = predictor.predict(text)
    orig_pred = add_probs(orig_pred)
    orig_probs = orig_pred['probs'][::-1]
    #print(orig_probs)
    if not selected_edits.empty:
        edited_input = selected_edits.iloc[0]["perturbed_text"]
    else:
        break
    if pd.isna(edited_input):
        break
    else:
        if i!=0:
            sublist.append([text, orig_probs])
        #display_classif_results(selected_edits)
        text = str(edited_input)

In [ ]:
import pickle
with open('imdb_textfooler_VERB.pickle', 'wb') as handle:
    pickle.dump(picklist, handle)

# For Mice and Polyjuice

In [ ]:
import pandas as pd
from tqdm import tqdm
picklist = []
edits = read_edits(f"{FOLDERS_PATH}/mice_imdb_VERB_0/edits.csv")
edits = get_best_edits(edits)
for index, row in tqdm(edits.iterrows()):
  orig_input = row['orig_input']
  orig_pred = predictor.predict(str(orig_input))
  orig_pred = add_probs(orig_pred)
  orig_probs = orig_pred['probs'][::-1]
  my_tupe = (orig_input, orig_probs)
  picklist.append([my_tupe])

In [ ]:
for sublist in tqdm(picklist):
  text = sublist[0][0]
  for i in range(10):
    edits = read_edits(f"{FOLDERS_PATH}/mice_imdb_VERB_{i}/edits.csv")
    edits = get_best_edits(edits) 
    selected_edits = edits.loc[edits["orig_input"] == text]
    #print(text)
    orig_pred = predictor.predict(text)
    orig_pred = add_probs(orig_pred)
    orig_probs = orig_pred['probs'][::-1]
    #print(orig_probs)
    edited_input = selected_edits.iloc[0]["edited_input"]
    
    if pd.isna(edited_input):
        break
    else:
        if i!=0:
            sublist.append((text, orig_probs))
        #display_classif_results(selected_edits)
        text = str(edited_input)

In [ ]:
import pickle
with open('imdb_mice_verb.pickle', 'wb') as handle:
    pickle.dump(picklist, handle)